In [2]:
# importing libraries.
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv 

In [12]:

def parse_reviews(card):
    reviews = card.find('a',{'class':'review-count sbold-Labels'})
    rv_str =reviews.text.strip().split()
    rv = rv_str
    if rv[0][-1]=='k':
        reviews = float(rv[0][1:-1])*1000
        return reviews
def parse_c_type_hq_age(card):
    try:
        c_type = card.find_all('p',class_='infoEntity sbold-list-header')[0].text.strip()
        hq = card.find_all('p',class_='infoEntity sbold-list-header')[1].text.strip().split()[0]
        age= card.find_all('p',class_='infoEntity sbold-list-header')[2].text.strip()
        total_emp = card.find_all('p',class_='infoEntity sbold-list-header')[3].text.strip()
    except:
        c_type = 'nan'
        hq = 'nan'
        age = 'nan'
        total_emp = 'nan'
    return c_type, hq, age, total_emp
def company_tags(card):
    tags = []
    company_tags = card.find_all('a', class_='ab_chip body-medium')
    for tag in company_tags:
        tags.append(tag.text.strip())
    return tags
def get_info(soup):
    all_data ={
        'Company Name':[],
        'Company Type':[],
        'Headquarter':[],
        'Ratings':[],
        'Reviews':[],
        'Compnay Tags':[],
        'Age':[],
        'Total Employee':[],
        'Total Jobs':[],
        # 'About':[]
    }
    cards = soup.find_all('div',{'class':'ab-company-result-card'})
    for c in range(len(cards)):
        company_name=(cards[c].find('h2').text.strip())
        rating=cards[c].find('p').text.strip()
        reviews = parse_reviews(cards[c])
        tags = company_tags(cards[c])
        tota_jobs = cards[c].find_all('span',class_='caption-subdued-large')[-1].text.strip()
        c_type, hq, age, total_emp = parse_c_type_hq_age(cards[c])
        # about  = cards[c].find('p', class_='description body-small').text.strip()[:300]
        all_data['Company Name'].append(company_name)
        all_data['Company Type'].append(c_type)
        all_data['Headquarter'].append(hq)
        all_data['Ratings'].append(rating)
        all_data['Reviews'].append(reviews)
        all_data['Compnay Tags'].append(tags)
        all_data['Age'].append(age)
        all_data['Total Employee'].append(total_emp)
        all_data['Total Jobs'].append(tota_jobs)
        # all_data['About'].append(about)
    return pd.DataFrame(all_data)
        
    

In [13]:
soups = []
for i in range(1,10): # range can be put up to 233
    multiple_ambition = requests.get(f'https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page={i}')
    soup = BeautifulSoup(multiple_ambition.text, 'html.parser')
    soups.append(soup)
d =[]
for i in soups:
    d.append(get_info(i))
df = pd.concat(d, ignore_index=True)

In [19]:
df.sample(7)

,Company Name,Company Type,Headquarter,Ratings,Reviews,Compnay Tags,Age,Total Employee,Total Jobs
186,Suzlon Group,Public,"Pune,Maharashtra",4.2,1900.0,"[Internet, Manufacturing, Power, Fortune India...",27 years old,5k-10k Employees (India),10
8,Tech Mahindra,Public,"Pune,Maharashtra",3.8,20100.0,"[Consulting, Analytics / KPO / Research, IT Se...",36 years old,50k-1 Lakh Employees (India),1.2k
217,Tata Teleservice...,Public,"Mumbai,Maharashtra",4.2,1700.0,"[Telecom / ISP, IT Services & Consulting, Fort...",26 years old,201-500 Employees (India),3
192,McDonald's,Private,"Chicago,Illinois",4.3,1900.0,"[Retail, Hotels & Restaurants, Forbes Global 2...",67 years old,5k-10k Employees (India),17
65,ITC,Public,"Kolkata,West",3.9,6900.0,"[FMCG, Textile & Apparel, Pulp & Paper, Printi...",47 years old,10k-50k Employees (India),24
140,ICICI Lombard Ge...,Public,"Mumbai,Maharashtra",3.9,2200.0,"[Financial Services, Insurance, Public, Mumbai...",21 years old,10k-50k Employees (India),75
35,BYJU'S,Private,"Bangalore,Karnataka",3.6,9800.0,"[Internet, Education / Training, e-Learning / ...",11 years old,1k-5k Employees (India),896
